In [1]:
import pandas as pd
import numpy as np
import pickle
import time
import copy
import datetime
import tqdm

In [ ]:
# with open('sleeping.pickle', 'rb') as f:
#     sleeping = pickle.load(f)

In [ ]:
# sleep_by_minutes_dict = {id: sleeping[id]['sleep_by_minutes'] for id in sleeping if not sleeping[id]['sleep_by_minutes'].empty}
# with open('sleep_by_minutes.pickle', 'wb') as f:
#     pickle.dump(sleep_by_minutes_dict, f)

In [ ]:
with open('sleep_by_minutes.pickle', 'rb') as f:
    sleep_by_minutes_dict = pickle.load(f)

In [ ]:
with open('heart_rate.pickle', 'rb') as f:
    hr_df = pickle.load(f)
hr_df.head()

In [ ]:
# Create 'Timestamp' column from 'globalTime'
hr_df['Timestamp'] = hr_df['globalTime'].apply(lambda x: pd.Timestamp(x, unit='s'))

In [ ]:
with open('heart_rate.pickle', 'wb') as f:
    pickle.dump(hr_df, f)

In [ ]:
# Add 'Timestamp' column to sleep_by_minutes.pickle file
for id in sleep_by_minutes_dict:
    if sleep_by_minutes_dict[id].empty:
        continue
    tmp_df = sleep_by_minutes_dict[id][['Date', 'Hour', 'Minute']]
    tmp_df = tmp_df['Date'].astype(str) + ' ' + tmp_df['Hour'].astype(str) + ':' + tmp_df['Minute'].astype(str)
    sleep_by_minutes_dict[id]['Timestamp'] = pd.to_datetime(tmp_df)

In [ ]:
with open('sleep_by_minutes.pickle', 'wb') as f:
    pickle.dump(sleep_by_minutes_dict, f)

In [ ]:
# Get the heart rate measurement for each sleep sample
# If the heart rate measurement is not available, get the average of the heart rate from 30s before and after the sleep sample.
# If that's not available, leave it as NaN

for id, user_sleep_df in tqdm.tqdm(sleep_by_minutes_dict_new.items()):
    if user_sleep_df.empty:
        continue

    # There are many duplicates in the dataframes, don't handle now just ignore them
    user_hr_df = hr_df[hr_df['userId'] == id].drop_duplicates(subset='globalTime').sort_values(by='Timestamp')
    # user_sleep_df = user_sleep_df.drop_duplicates(subset='Timestamp').sort_values(by='Timestamp')
    sleep_df_dups = user_sleep_df.duplicated(subset='Timestamp')

    # First, get the 'pulse' value of all entries with timestamp that exist in user_hr_df
    sleep_ts_mask = (user_sleep_df['Timestamp'].isin(user_hr_df['Timestamp'])) & (~sleep_df_dups)
    hr_ts_mask = user_hr_df['Timestamp'].isin(user_sleep_df['Timestamp'])
    assert sleep_ts_mask.sum() == hr_ts_mask.sum()  # make sure the next line is valid
    # Assigning to the original df (not the copy)
    sleep_by_minutes_dict_new[id].loc[sleep_ts_mask, 'Pulse'] = \
        user_hr_df.loc[hr_ts_mask, 'pulse'].to_numpy(dtype=int, copy=True)

    # Next, handle other entries
    # next_hr_time = user_hr_df.iloc[0]['Timestamp']
    for i, row in user_sleep_df[~sleep_ts_mask].iterrows():
        timestamp = row['Timestamp']
        # if abs(timestamp - next_hr_time) > pd.Timedelta('30s'):
        #     if timestamp > next_hr_time:
        #         tmp_hr_df = user_hr_df[user_hr_df['Timestamp'] >= timestamp]
        #         if tmp_hr_df.empty:
        #             break
        #         next_hr_time = tmp_hr_df.iloc[0]['Timestamp']
        #     continue
        hr_df_slice = user_hr_df[(user_hr_df['Timestamp'] >= (timestamp - pd.Timedelta('30s'))) &
                                 (user_hr_df['Timestamp'] <= (timestamp + pd.Timedelta('30s')))]
        if hr_df_slice.empty:
            continue
        if timestamp in hr_df_slice['Timestamp'].values:
            sleep_by_minutes_dict_new[id].loc[i, 'Pulse'] = \
                hr_df_slice[hr_df_slice['Timestamp'] == timestamp]['pulse'].values[0]
        else:
            sleep_by_minutes_dict_new[id].loc[i, 'Pulse'] = hr_df_slice['pulse'].mean()
        # next_hr_time = user_hr_df[user_hr_df['Timestamp'] >= timestamp].iloc[0]['Timestamp']


In [ ]:
def func(id):
    user_sleep_df = sleep_by_minutes_dict_new[id].copy()

    if user_sleep_df.empty:
        return 0

    # There are many duplicates in the dataframes, don't handle now just ignore them
    user_hr_df = hr_df[hr_df['userId'] == id].drop_duplicates(subset='globalTime').sort_values(by='Timestamp')
    sleep_df_dups = user_sleep_df.duplicated(subset='Timestamp')

    # First, get the 'pulse' value of all entries with timestamp that exist in user_hr_df
    sleep_ts_mask = (user_sleep_df['Timestamp'].isin(user_hr_df['Timestamp'])) & (~sleep_df_dups)
    hr_ts_mask = user_hr_df['Timestamp'].isin(user_sleep_df['Timestamp'])
    assert sleep_ts_mask.sum() == hr_ts_mask.sum()  # make sure the next line is valid

    # Assigning to the original df (not the copy)
    sleep_by_minutes_dict_new[id].loc[sleep_ts_mask, 'Pulse'] = \
        user_hr_df.loc[hr_ts_mask, 'pulse'].to_numpy(dtype=int, copy=True)

    # Next, handle other entries
    for i, row in user_sleep_df[~sleep_ts_mask].iterrows():
        timestamp = row['Timestamp']
        hr_df_slice = user_hr_df[(user_hr_df['Timestamp'] >= (timestamp - pd.Timedelta('30s'))) &
                                 (user_hr_df['Timestamp'] <= (timestamp + pd.Timedelta('30s')))]
        if hr_df_slice.empty:
            continue
        if timestamp in hr_df_slice['Timestamp'].values:
            sleep_by_minutes_dict_new[id].loc[i, 'Pulse'] = \
                hr_df_slice[hr_df_slice['Timestamp'] == timestamp]['pulse'].values[0]
        else:
            sleep_by_minutes_dict_new[id].loc[i, 'Pulse'] = hr_df_slice['pulse'].mean()
    return 0

from multiprocessing import Pool

if __name__ == '__main__':
    with Pool() as pool:
        count = 0
        for _ in tqdm.tqdm(pool.imap_unordered(func, list(sleep_by_minutes_dict_new.keys()))):
            count += 1
            print(f'Done {100 * count / len(sleep_by_minutes_dict_new):.2f}%')


In [ ]:
user_sleep_df

In [ ]:
user_hr_df

In [ ]:
# count = 0
# user_sleep_df = sleep_by_minutes_dict_new[id].copy()
# user_hr_df = hr_df[hr_df['userId'] == id].copy()

# There are many duplicates in the hr_df
user_hr_df.drop_duplicates(subset='globalTime', inplace=True)
user_sleep_df.drop_duplicates(subset='Timestamp', inplace=True)

# Handle timestamp entries from user_sleep_df that are in user_hr_df
sleep_ts_mask = user_sleep_df['Timestamp'].isin(user_hr_df['Timestamp'])
hr_ts_mask = user_hr_df['Timestamp'].isin(user_sleep_df['Timestamp'])
assert sleep_ts_mask.sum() == hr_ts_mask.sum()
user_sleep_df.loc[sleep_ts_mask, 'Pulse'] = user_hr_df.loc[hr_ts_mask, 'pulse'].to_numpy(dtype=int, copy=True)

next_hr_time = max(user_hr_df.iloc[0]['Timestamp'], user_sleep_df.iloc[0]['Timestamp'])
for i, row in user_sleep_df[~sleep_ts_mask].iterrows():
    timestamp = row['Timestamp']
    if abs(timestamp - next_hr_time) > pd.Timedelta('30s'):
        if timestamp > next_hr_time:
            next_hr_time = user_hr_df[user_hr_df['Timestamp'] >= timestamp].iloc[0]['Timestamp']
        continue
    hr_df_slice = user_hr_df[(user_hr_df['Timestamp'] >= (timestamp - pd.Timedelta('30s'))) &
                             (user_hr_df['Timestamp'] <= (timestamp + pd.Timedelta('30s')))]
    if hr_df_slice.empty:
        continue
    if timestamp in hr_df_slice['Timestamp'].values:
        user_sleep_df.loc[i, 'Pulse'] = hr_df_slice[hr_df_slice['Timestamp'] == timestamp]['pulse'].values[0]
    else:
        user_sleep_df.loc[i, 'Pulse'] = hr_df_slice['pulse'].mean()
    next_hr_time = user_hr_df[user_hr_df['Timestamp'] >= timestamp].iloc[0]['Timestamp']
    count += 1